# Postprocess the response data from the OpenAI API

input: `respose_file_path`
output: `annotated_file_path`

Description
- make the annotated dataset from the response from the OpenAI API.

In [1]:
import pandas as pd
from datasets import Features, ClassLabel, Dataset, DatasetDict
import json
import re

/Users/jangmin/anaconda3/envs/metal/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
respose_file_path = r'../local_data/api_responses_for_annotated_dataset.jsonl'
annotated_file_path = r'../local_data/annotated_dataset.pkl'

In [4]:
in_fname =  respose_file_path
datas = []
request_ids = []
# results = {}
with open(in_fname, 'r') as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        if isinstance(data[1] , dict):
            # results[i] = data
            datas.append(data[1]['choices'][0]['message']['content'])
            request_ids.append(data[2]['request_id'])

In [6]:
df = pd.DataFrame({'request_id': request_ids, 'response': datas})
df['response'] = df['response'].str.split('\n')
df = df.explode('response')
df.sort_values(by='request_id', inplace=True)

In [7]:
df[['source', 'target']] = df['response'].str.split(' -> ', expand=True)

In [ ]:
df['target'] = df['target'].fillna('')
df = df[df.source.str.len() > 0]

In [25]:
df[['source', 'target']].to_pickle(annotated_file_path)